In [1]:
import opendatasets as od 
import numpy as np 
import pandas as pd 
import os 

In [10]:
files = od.download('https://www.kaggle.com/datasets/willianoliveiragibin/healthcare-insurance')
dir = '.\healthcare-insurance'
file = os.listdir(dir)
file_csv=file[0]
path = os.path.join(dir,file_csv)
df = pd.read_csv(path)
df.head(5)

Skipping, found downloaded files in ".\healthcare-insurance" (use force=True to force download)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [12]:
df.describe

<bound method NDFrame.describe of       age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]>

In [16]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [26]:
dummies = pd.get_dummies(df[['sex','smoker','region']],drop_first=True)
dummies = dummies.astype(int)
print(dummies)

      sex_male  smoker_yes  region_northwest  region_southeast  \
0            0           1                 0                 0   
1            1           0                 0                 1   
2            1           0                 0                 1   
3            1           0                 1                 0   
4            1           0                 1                 0   
...        ...         ...               ...               ...   
1333         1           0                 1                 0   
1334         0           0                 0                 0   
1335         0           0                 0                 1   
1336         0           0                 0                 0   
1337         0           1                 1                 0   

      region_southwest  
0                    1  
1                    0  
2                    0  
3                    0  
4                    0  
...                ...  
1333                 0  
1334   

In [33]:
df_2 = pd.concat([df,dummies],axis=1)
df_2.drop(columns={'sex','smoker','region'},inplace=True)
df_2

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0,1,0,0
1334,18,31.920,0,2205.98080,0,0,0,0,0
1335,18,36.850,0,1629.83350,0,0,0,1,0
1336,21,25.800,0,2007.94500,0,0,0,0,1


In [55]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model 
import statsmodels.api as sm
from sklearn.metrics import r2_score

In [35]:
correlation = df_2.corr()['charges']
correlation

age                 0.299008
bmi                 0.198341
children            0.067998
charges             1.000000
sex_male            0.057292
smoker_yes          0.787251
region_northwest   -0.039905
region_southeast    0.073982
region_southwest   -0.043210
Name: charges, dtype: float64

In [37]:
independent_variables = df_2[['age','bmi','sex_male','smoker_yes','region_northwest','region_southeast','region_southwest']]
independent_variables

,age,bmi,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,0,0,1
1,18,33.770,1,0,0,1,0
2,28,33.000,1,0,0,1,0
3,33,22.705,1,0,1,0,0
4,32,28.880,1,0,1,0,0
...,...,...,...,...,...,...,...
1333,50,30.970,1,0,1,0,0
1334,18,31.920,0,0,0,0,0
1335,18,36.850,0,0,0,1,0
1336,21,25.800,0,0,0,0,1


In [39]:
target_variable = df_2['charges']
target_variable

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

In [41]:
x = independent_variables.to_numpy()
y= target_variable

In [57]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [65]:
lr= linear_model.LinearRegression()
lr

LinearRegression()

In [66]:
lr.fit(x_train,y_train)

LinearRegression()

In [60]:
coefficient = lr.coef_
coefficient 

array([ 2.59260167e+02,  3.36572870e+02, -2.17545814e+00,  2.36737435e+04,
       -3.36820721e+02, -6.73822153e+02, -8.08196184e+02])

In [61]:
intercept = lr.intercept_
intercept

-11551.59310267684

In [80]:
prediction= lr.predict(df_2[['age','bmi','sex_male','smoker_yes','region_northwest','region_southeast','region_southwest']])
prediction

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([25630.28046248,  3805.15810879,  6138.59866911, ...,
        4843.97800619,  1768.25426422, 37384.37319756])

In [82]:
df_2['prediction'] = prediction
df_2.reset_index(drop=True, inplace=True)
df_2

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest,prediction
0,19,27.900,0,16884.92400,0,1,0,0,1,25630.280462
1,18,33.770,1,1725.55230,1,0,0,1,0,3805.158109
2,28,33.000,3,4449.46200,1,0,0,1,0,6138.598669
3,33,22.705,0,21984.47061,1,0,1,0,0,4306.883241
4,32,28.880,0,3866.85520,1,0,1,0,0,6125.960545
...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0,1,0,0,11496.080850
1334,18,31.920,0,2205.98080,0,0,0,0,0,3858.495910
1335,18,36.850,0,1629.83350,0,0,0,1,0,4843.978006
1336,21,25.800,0,2007.94500,0,0,0,0,1,1768.254264


In [75]:
coefficient_detemination = r2_score(y_train,prediction)
coefficient_detemination

0.7398848762300241

In [85]:
model = sm.OLS(y_train,sm.add_constant(x_train))
result = model.fit()
summary = result.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.740
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                     431.5
Date:                Sun, 22 Oct 2023   Prob (F-statistic):          2.64e-305
Time:                        17:56:59   Log-Likelihood:                -10849.
No. Observations:                1070   AIC:                         2.171e+04
Df Residuals:                    1062   BIC:                         2.175e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.155e+04   1109.333    -10.413      0.0